In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Outrights/LE Feb25_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=12).mean()

# Initialize variables
entry_points = []
exit_points = []
positions = []
trades = []
daily_returns = []

# DataFrame to store trade details
trade_details = pd.DataFrame(columns=['Entry Date', 'Exit Date', 'Trade Type', 'Entry Price', 'Exit Price', 'Profit'])

# Constants for tick size and tick price
tick_size = 0.025
tick_price = 10

# Initialize variables to track the current position
current_position = None
current_entry_price = None
current_entry_date = None

# Iterate over DataFrame to backtest strategy
for i in range(1, len(df)):
    daily_return = 0

    # Check for crossing points (sign change in the difference)
    if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:
        # MA9 crosses above MA12
        if current_position == 'short':
            # Exit short position
            exit_price = df['Close'][i]
            ticks = (current_entry_price - exit_price) / tick_size
            profit = ticks * tick_price
            exit_points.append((df['Serial'][i], exit_price, 'short'))
            trades.append(profit)
            trade_details = pd.concat([trade_details, pd.DataFrame({
                'Entry Date': [current_entry_date],
                'Exit Date': [df['Date'][i]],
                'Trade Type': ['Short'],
                'Entry Price': [current_entry_price],
                'Exit Price': [exit_price],
                'Profit': [profit]
            })], ignore_index=True)
            daily_return = profit / current_entry_price
            current_position = None
        
        # Open long position
        current_position = 'long'
        current_entry_price = df['Close'][i]
        current_entry_date = df['Date'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
    
    elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:
        # MA9 crosses below MA12
        if current_position == 'long':
            # Exit long position
            exit_price = df['Close'][i]
            ticks = (exit_price - current_entry_price) / tick_size
            profit = ticks * tick_price
            exit_points.append((df['Serial'][i], exit_price, 'long'))
            trades.append(profit)
            trade_details = pd.concat([trade_details, pd.DataFrame({
                'Entry Date': [current_entry_date],
                'Exit Date': [df['Date'][i]],
                'Trade Type': ['Long'],
                'Entry Price': [current_entry_price],
                'Exit Price': [exit_price],
                'Profit': [profit]
            })], ignore_index=True)
            daily_return = profit / current_entry_price
            current_position = None
        
        # Open short position
        current_position = 'short'
        current_entry_price = df['Close'][i]
        current_entry_date = df['Date'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'short'))

    # Check for trend reversal to exit the current position
    if current_position == 'long':
        if df['Low'][i] < min(df['Low'][:i]) or df['High'][i] > max(df['High'][:i]):
            # Exit long position if new low is made
            exit_price = df['Close'][i]
            ticks = (exit_price - current_entry_price) / tick_size
            profit = ticks * tick_price
            exit_points.append((df['Serial'][i], exit_price, 'long'))
            trades.append(profit)
            trade_details = pd.concat([trade_details, pd.DataFrame({
                'Entry Date': [current_entry_date],
                'Exit Date': [df['Date'][i]],
                'Trade Type': ['Long'],
                'Entry Price': [current_entry_price],
                'Exit Price': [exit_price],
                'Profit': [profit]
            })], ignore_index=True)
            daily_return = profit / current_entry_price
            current_position = None
    
    if current_position == 'short':
        if df['High'][i] > max(df['High'][:i]) or df['Low'][i] < min(df['Low'][:i]):
            # Exit short position if new high is made
            exit_price = df['Close'][i]
            ticks = (current_entry_price - exit_price) / tick_size
            profit = ticks * tick_price
            exit_points.append((df['Serial'][i], exit_price, 'short'))
            trades.append(profit)
            trade_details = pd.concat([trade_details, pd.DataFrame({
                'Entry Date': [current_entry_date],
                'Exit Date': [df['Date'][i]],
                'Trade Type': ['Short'],
                'Entry Price': [current_entry_price],
                'Exit Price': [exit_price],
                'Profit': [profit]
            })], ignore_index=True)
            daily_return = profit / current_entry_price
            current_position = None
    
    daily_returns.append(daily_return)

# Close any open position at the end of the period
if current_position:
    exit_price = df['Close'].iloc[-1]
    if current_position == 'long':
        ticks = (exit_price - current_entry_price) / tick_size
    else:
        ticks = (current_entry_price - exit_price) / tick_size
    profit = ticks * tick_price
    exit_points.append((df['Serial'].iloc[-1], exit_price, current_position))
    trades.append(profit)
    trade_details = pd.concat([trade_details, pd.DataFrame({
        'Entry Date': [current_entry_date],
        'Exit Date': [df['Date'].iloc[-1]],
        'Trade Type': [current_position.capitalize()],
        'Entry Price': [current_entry_price],
        'Exit Price': [exit_price],
        'Profit': [profit]
    })], ignore_index=True)

# Calculate total profit
total_profit = sum(trades)

# Calculate Sharpe Ratio
risk_free_rate = 0.02  # Assume 2% annual risk-free rate
daily_rf_rate = (1 + risk_free_rate) ** (1/252) - 1
excess_returns = np.array(daily_returns) - daily_rf_rate
sharpe_ratio = np.sqrt(252) * np.mean(excess_returns) / np.std(excess_returns)

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA12', line=dict(color='red')))

# Add entry and exit points to the chart
for point in entry_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Entry {point[2]}", 
                             marker=dict(symbol='triangle-up' if point[2] == 'long' else 'triangle-down', size=10, color='green' if point[2] == 'long' else 'red')))
for point in exit_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Exit {point[2]}", 
                             marker=dict(symbol='x', size=12, color='blue')))

# Update layout
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

# Print performance metrics
print(f"Total Profit: ${total_profit:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

# Display trade details
print(trade_details)


Total Profit: $-12420.00
Sharpe Ratio: -1.74
   Entry Date  Exit Date Trade Type  Entry Price  Exit Price  Profit
0  2023-09-29 2023-10-17      Short      196.025     196.375  -140.0
1  2023-10-17 2023-10-20       Long      196.375     192.500 -1550.0
2  2023-10-20 2023-10-23      Short      192.500     188.750  1500.0
3  2023-11-06 2023-11-07       Long      190.350     186.900 -1380.0
4  2023-11-10 2023-11-10      Short      184.000     184.000     0.0
5  2023-11-24 2023-11-24       Long      183.025     183.025     0.0
6  2023-11-28 2023-11-28      Short      182.975     182.975     0.0
7  2023-12-19 2024-01-17       Long      182.500     185.275  1110.0
8  2024-01-17 2024-01-22      Short      185.275     186.950  -670.0
9  2024-01-22 2024-02-15       Long      186.950     189.900  1180.0
10 2024-02-15 2024-02-28      Short      189.900     191.350  -580.0
11 2024-02-28 2024-03-11       Long      191.350     192.425   430.0
12 2024-03-11 2024-03-12      Short      192.425     192.9

In [55]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Function to calculate strategy performance based on given MAs
def backtest_strategy(ma_short, ma_long):
    # Calculate Moving Averages
    df[f'MA{ma_short}'] = df['Close'].rolling(window=ma_short).mean()
    df[f'MA{ma_long}'] = df['Close'].rolling(window=ma_long).mean()

    # Initialize variables
    entry_points = []
    exit_points = []
    positions = []
    trades = []
    daily_returns = []

    # DataFrame to store trade details
    trade_details = pd.DataFrame(columns=['Entry Date', 'Exit Date', 'Trade Type', 'Entry Price', 'Exit Price', 'Profit', 'Exit Reason'])

    # Constants for tick size and tick price
    tick_size = 0.25
    tick_price = 12.5

    # Iterate over DataFrame to backtest strategy
    for i in range(1, len(df)):
        daily_return = 0
        
        # Check for crossing points (sign change in the difference)
        if df[f'MA{ma_short}'][i] > df[f'MA{ma_long}'][i] and df[f'MA{ma_short}'][i-1] <= df[f'MA{ma_long}'][i-1]:
            # MA short crosses above MA long
            if positions and positions[-1]['type'] == 'long':
                # Close long position
                exit_price = df['Close'][i]
                entry_price = positions[-1]['entry_price']
                ticks = (exit_price - entry_price) / tick_size
                profit = ticks * tick_price
                exit_points.append((df['Serial'][i], exit_price, 'long'))
                trades.append(profit)
                trade_details = pd.concat([trade_details, pd.DataFrame({
                    'Entry Date': [positions[-1]['entry_date']],
                    'Exit Date': [df['Date'][i]],
                    'Trade Type': ['Long'],
                    'Entry Price': [entry_price],
                    'Exit Price': [exit_price],
                    'Profit': [profit],
                    'Exit Reason': ['MA Crossover']
                })], ignore_index=True)
                daily_return = profit / entry_price
                positions.pop()
            
            # Open short position
            positions.append({'type': 'short', 'entry_price': df['Close'][i], 'entry_date': df['Date'][i]})
            entry_points.append((df['Serial'][i], df['Close'][i], 'short'))
        
        elif df[f'MA{ma_short}'][i] < df[f'MA{ma_long}'][i] and df[f'MA{ma_short}'][i-1] >= df[f'MA{ma_long}'][i-1]:
            # MA short crosses below MA long
            if positions and positions[-1]['type'] == 'short':
                # Close short position
                exit_price = df['Close'][i]
                entry_price = positions[-1]['entry_price']
                ticks = (entry_price - exit_price) / tick_size
                profit = ticks * tick_price
                exit_points.append((df['Serial'][i], exit_price, 'short'))
                trades.append(profit)
                trade_details = pd.concat([trade_details, pd.DataFrame({
                    'Entry Date': [positions[-1]['entry_date']],
                    'Exit Date': [df['Date'][i]],
                    'Trade Type': ['Short'],
                    'Entry Price': [entry_price],
                    'Exit Price': [exit_price],
                    'Profit': [profit],
                    'Exit Reason': ['MA Crossover']
                })], ignore_index=True)
                daily_return = profit / entry_price
                positions.pop()
            
            # Open long position
            positions.append({'type': 'long', 'entry_price': df['Close'][i], 'entry_date': df['Date'][i]})
            entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
        
        # Check for trend break (new low for long positions, new high for short positions)
        elif positions:
            if positions[-1]['type'] == 'long' and df['Low'][i] < df['Low'][i-1]:
                # Close long position due to trend break
                exit_price = df['Low'][i]  # Use the low price as exit price
                entry_price = positions[-1]['entry_price']
                ticks = (exit_price - entry_price) / tick_size
                profit = ticks * tick_price
                exit_points.append((df['Serial'][i], exit_price, 'long'))
                trades.append(profit)
                trade_details = pd.concat([trade_details, pd.DataFrame({
                    'Entry Date': [positions[-1]['entry_date']],
                    'Exit Date': [df['Date'][i]],
                    'Trade Type': ['Long'],
                    'Entry Price': [entry_price],
                    'Exit Price': [exit_price],
                    'Profit': [profit],
                    'Exit Reason': ['Trend Break']
                })], ignore_index=True)
                daily_return = profit / entry_price
                positions.pop()
            elif positions[-1]['type'] == 'short' and df['High'][i] > df['High'][i-1]:
                # Close short position due to trend break
                exit_price = df['High'][i]  # Use the high price as exit price
                entry_price = positions[-1]['entry_price']
                ticks = (entry_price - exit_price) / tick_size
                profit = ticks * tick_price
                exit_points.append((df['Serial'][i], exit_price, 'short'))
                trades.append(profit)
                trade_details = pd.concat([trade_details, pd.DataFrame({
                    'Entry Date': [positions[-1]['entry_date']],
                    'Exit Date': [df['Date'][i]],
                    'Trade Type': ['Short'],
                    'Entry Price': [entry_price],
                    'Exit Price': [exit_price],
                    'Profit': [profit],
                    'Exit Reason': ['Trend Break']
                })], ignore_index=True)
                daily_return = profit / entry_price
                positions.pop()
        
        daily_returns.append(daily_return)

    # Close any open position at the end of the period
    if positions:
        exit_price = df['Close'].iloc[-1]
        entry_price = positions[-1]['entry_price']
        if positions[-1]['type'] == 'long':
            ticks = (exit_price - entry_price) / tick_size
        else:
            ticks = (entry_price - exit_price) / tick_size
        profit = ticks * tick_price
        exit_points.append((df['Serial'].iloc[-1], exit_price, positions[-1]['type']))
        trades.append(profit)
        trade_details = pd.concat([trade_details, pd.DataFrame({
            'Entry Date': [positions[-1]['entry_date']],
            'Exit Date': [df['Date'].iloc[-1]],
            'Trade Type': [positions[-1]['type'].capitalize()],
            'Entry Price': [entry_price],
            'Exit Price': [exit_price],
            'Profit': [profit],
            'Exit Reason': ['End of Period']
        })], ignore_index=True)

    # Calculate total profit
    total_profit = sum(trades)

    # Calculate Sharpe Ratio
    risk_free_rate = 0.02  # Assume 2% annual risk-free rate
    daily_rf_rate = (1 + risk_free_rate) ** (1/252) - 1
    excess_returns = np.array(daily_returns) - daily_rf_rate
    sharpe_ratio = np.sqrt(252) * np.mean(excess_returns) / np.std(excess_returns)

    return total_profit, sharpe_ratio, trade_details
# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/ZW Outright/ZW Dec24_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Constants for tick size and tick price
tick_size = 0.025
tick_price = 10

# Define ranges for moving averages
ma_range = range(5, 21)

# Variables to store the best results
best_ma_pair = (0, 0)
best_profit = -np.inf
best_sharpe = -np.inf
best_entry_points = []
best_exit_points = []

# Loop through combinations of short and long MAs
for ma_short in ma_range:
    for ma_long in ma_range:
        if ma_long > ma_short:  # Ensure MA long is greater than MA short
            total_profit, sharpe_ratio, trade_details = backtest_strategy(ma_short, ma_long)
            if total_profit > best_profit:
                best_profit = total_profit
                best_sharpe = sharpe_ratio
                best_ma_pair = (ma_short, ma_long)
                best_trade_details = trade_details

# Print best MA pair and performance metrics
print(f"Best MA Pair: MA{best_ma_pair[0]} and MA{best_ma_pair[1]}")
print(f"Total Profit: ${best_profit:.2f}")
print(f"Sharpe Ratio: {best_sharpe:.2f}")

best_trade_details['Entry Date'] = pd.to_datetime(best_trade_details['Entry Date'])
best_trade_details['Exit Date'] = pd.to_datetime(best_trade_details['Exit Date'])

# Calculate the number of days between Entry Date and Exit Date
best_trade_details['Days Between'] = (best_trade_details['Exit Date'] - best_trade_details['Entry Date']).dt.days

# Print the updated DataFrame
best_trade_details1 = best_trade_details
best_trade_details1

Best MA Pair: MA19 and MA20
Total Profit: $1637.50
Sharpe Ratio: 0.18


,Entry Date,Exit Date,Trade Type,Entry Price,Exit Price,Profit,Exit Reason,Days Between
0,2023-02-21,2023-03-15,Short,783.50,733.25,2512.5,MA Crossover,22
1,2023-03-15,2023-03-16,Long,733.25,727.50,-287.5,Trend Break,1
2,2023-04-12,2023-04-13,Short,722.50,713.25,462.5,MA Crossover,1
3,2023-04-13,2023-04-14,Long,713.25,719.75,325.0,MA Crossover,1
4,2023-04-14,2023-04-17,Short,719.75,733.00,-662.5,MA Crossover,3
5,2023-04-17,2023-04-20,Long,733.00,715.75,-862.5,MA Crossover,3
6,2023-04-20,2023-04-24,Short,715.75,716.00,-12.5,Trend Break,4
7,2023-04-26,2023-04-27,Long,703.25,691.25,-600.0,Trend Break,1
8,2023-05-18,2023-05-19,Short,685.00,679.75,262.5,MA Crossover,1
9,2023-05-19,2023-05-22,Long,679.75,675.00,-237.5,Trend Break,3


In [49]:
best_trade_details1['Days Between'].median()

1.5

In [3]:
# Approach with single exit condition of new entry then exit
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Function to calculate strategy performance based on given MAs
def backtest_strategy(ma_short, ma_long):
    # Calculate Moving Averages
    df[f'MA9'] = df['Close'].rolling(window=ma_short).mean()
    df[f'MA12'] = df['Close'].rolling(window=ma_long).mean()

    # Initialize variables
    entry_points = []
    exit_points = []
    positions = []
    trades = []
    daily_returns = []

    # DataFrame to store trade details
    trade_details = pd.DataFrame(columns=['Entry Date', 'Exit Date', 'Trade Type', 'Entry Price', 'Exit Price', 'Profit'])

    # Constants for tick size and tick price
    tick_size = 0.025
    tick_price = 10

    # Iterate over DataFrame to backtest strategy
    for i in range(1, len(df)):
        daily_return = 0
        
        # Check for crossing points (sign change in the difference)
        if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:
            # MA9 crosses above MA12
            if positions and positions[-1]['type'] == 'long':
                # Close long position
                exit_price = df['Close'][i]
                entry_price = positions[-1]['entry_price']
                ticks = (exit_price - entry_price) / tick_size
                profit = ticks * tick_price
                exit_points.append((df['Serial'][i], exit_price, 'long'))
                trades.append(profit)
                trade_details = pd.concat([trade_details, pd.DataFrame({
                    'Entry Date': [positions[-1]['entry_date']],
                    'Exit Date': [df['Date'][i]],
                    'Trade Type': ['Long'],
                    'Entry Price': [entry_price],
                    'Exit Price': [exit_price],
                    'Profit': [profit]
                })], ignore_index=True)
                daily_return = profit / entry_price
                positions.pop()
            
            # Open short position
            positions.append({'type': 'short', 'entry_price': df['Close'][i], 'entry_date': df['Date'][i]})
            entry_points.append((df['Serial'][i], df['Close'][i], 'short'))
        
        elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:
            # MA9 crosses below MA12
            if positions and positions[-1]['type'] == 'short':
                # Close short position
                exit_price = df['Close'][i]
                entry_price = positions[-1]['entry_price']
                ticks = (entry_price - exit_price) / tick_size
                profit = ticks * tick_price
                exit_points.append((df['Serial'][i], exit_price, 'short'))
                trades.append(profit)
                trade_details = pd.concat([trade_details, pd.DataFrame({
                    'Entry Date': [positions[-1]['entry_date']],
                    'Exit Date': [df['Date'][i]],
                    'Trade Type': ['Short'],
                    'Entry Price': [entry_price],
                    'Exit Price': [exit_price],
                    'Profit': [profit]
                })], ignore_index=True)
                daily_return = profit / entry_price
                positions.pop()
            
            # Open long position
            positions.append({'type': 'long', 'entry_price': df['Close'][i], 'entry_date': df['Date'][i]})
            entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
        
        daily_returns.append(daily_return)

    # Close any open position at the end of the period
    if positions:
        exit_price = df['Close'].iloc[-1]
        entry_price = positions[-1]['entry_price']
        if positions[-1]['type'] == 'long':
            ticks = (exit_price - entry_price) / tick_size
        else:
            ticks = (entry_price - exit_price) / tick_size
        profit = ticks * tick_price
        exit_points.append((df['Serial'].iloc[-1], exit_price, positions[-1]['type']))
        trades.append(profit)
        trade_details = pd.concat([trade_details, pd.DataFrame({
            'Entry Date': [positions[-1]['entry_date']],
            'Exit Date': [df['Date'].iloc[-1]],
            'Trade Type': [positions[-1]['type'].capitalize()],
            'Entry Price': [entry_price],
            'Exit Price': [exit_price],
            'Profit': [profit]
        })], ignore_index=True)

    # Calculate total profit
    total_profit = sum(trades)

    # Calculate Sharpe Ratio
    risk_free_rate = 0.02  # Assume 2% annual risk-free rate
    daily_rf_rate = (1 + risk_free_rate) ** (1/252) - 1
    excess_returns = np.array(daily_returns) - daily_rf_rate
    sharpe_ratio = np.sqrt(252) * np.mean(excess_returns) / np.std(excess_returns)
    

    return total_profit, sharpe_ratio, trade_details

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25_90min.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Constants for tick size and tick price
tick_size = 0.25
tick_price = 12.5

# Define ranges for moving averages
ma_range = range(5, 21)

# Variables to store the best results
best_ma_pair = (0, 0)
best_profit = -np.inf
best_sharpe = -np.inf
best_entry_points = []
best_exit_points = []

# Loop through combinations of short and long MAs
for ma_short in ma_range:
    for ma_long in ma_range:
        if ma_long > ma_short:  # Ensure MA long is greater than MA short
            total_profit, sharpe_ratio, trade_details = backtest_strategy(ma_short, ma_long)
            if total_profit > best_profit:
                best_profit = total_profit
                best_sharpe = sharpe_ratio
                best_ma_pair = (ma_short, ma_long)
                best_trade_details = trade_details

# Print best MA pair and performance metrics
print(f"Best MA Pair: MA{best_ma_pair[0]} and MA{best_ma_pair[1]}")
print(f"Total Profit: ${best_profit:.2f}")
print(f"Sharpe Ratio: {best_sharpe:.2f}")

best_trade_details['Entry Date'] = pd.to_datetime(best_trade_details['Entry Date'])
best_trade_details['Exit Date'] = pd.to_datetime(best_trade_details['Exit Date'])

# Calculate the number of days between Entry Date and Exit Date
best_trade_details['Days Between'] = (best_trade_details['Exit Date'] - best_trade_details['Entry Date']).dt.days

# Print the updated DataFrame
best_trade_details

Best MA Pair: MA10 and MA12
Total Profit: $6960.00
Sharpe Ratio: 0.49


,Entry Date,Exit Date,Trade Type,Entry Price,Exit Price,Profit,Days Between
0,2023-09-20 15:00:00,2023-09-21 13:30:00,Long,199.175,198.575,-240.0,0
1,2023-09-21 13:30:00,2023-09-21 15:00:00,Short,198.575,198.325,100.0,0
2,2023-09-21 15:00:00,2023-09-22 16:30:00,Long,198.325,199.150,330.0,1
3,2023-09-22 16:30:00,2023-09-25 13:30:00,Short,199.150,198.600,220.0,2
4,2023-09-25 13:30:00,2023-09-27 13:30:00,Long,198.600,197.575,-410.0,2
...,...,...,...,...,...,...,...
110,2024-08-16 16:30:00,2024-08-23 15:00:00,Long,179.150,177.200,-780.0,6
111,2024-08-23 15:00:00,2024-08-29 18:00:00,Short,177.200,177.300,-40.0,6
112,2024-08-29 18:00:00,2024-09-03 16:30:00,Long,177.300,179.675,950.0,4
113,2024-09-03 16:30:00,2024-09-05 18:00:00,Short,179.675,177.275,960.0,2


In [51]:
best_trade_details['Days Between'].median()

5.5